In [ ]:
# ref: https://coolpuppy.readthedocs.io/en/latest/Examples/Walkthrough_API.html

In [ ]:
import os
print(os.getcwd())
os.chdir("/data1/sunwenju/projects_on_940xa/2024041703_mESC_HiC_use_pairtools_cooler_pipeline")
print(os.getcwd())

In [ ]:
! find . -name *rep12.merged.1k.zoomify.mcool

In [ ]:
! find . -name *WT*rep1.*1k.zoomify.mcool

In [ ]:
! find . -name *WT*rep2.*1k.zoomify.mcool

# 读取需要用到的文件

## 读取 mcool 文件

In [ ]:
# 读取 mcool 文件
import cooler
resolution = 10_000
# mcool 文件
nora_wt_mcool_file = "./Nora_WT/Nora_mESC_WT_HiC.rep12.merged.1k.zoomify.mcool"
nora_iaa_mcool_file = "./Nora_IAA/Nora_mESC_IAA_HiC.rep12.merged.1k.zoomify.mcool"
ren_wt_mcool_file = "./Ren_WT/Ren_mESC_WT_HiC.rep12.merged.1k.zoomify.mcool"
ren_iaa_mcool_file = "./Ren_IAA/Ren_mESC_IAA_HiC.rep12.merged.1k.zoomify.mcool"
wn_bpk25_mcool_file = "./WN_BPK25/WN_mESC_BPK25_HiC.rep12.merged.1k.zoomify.mcool"
# 生成 r10k cool 互作矩阵对象
nora_wt_r10k_clr = cooler.Cooler(f'{nora_wt_mcool_file}::/resolutions/{resolution}')
nora_iaa_r10k_clr = cooler.Cooler(f'{nora_iaa_mcool_file}::/resolutions/{resolution}')
ren_wt_r10k_clr = cooler.Cooler(f'{ren_wt_mcool_file}::/resolutions/{resolution}')
ren_iaa_r10k_clr = cooler.Cooler(f'{ren_iaa_mcool_file}::/resolutions/{resolution}')
wn_bpk25_r10k_clr = cooler.Cooler(f'{wn_bpk25_mcool_file}::/resolutions/{resolution}')


In [ ]:
# Nora/Ren WT rep1/2 mcool 后续用于 divide pileup 表示 WT 两次重复之间没啥差异，用 IAA 或 BPK25 处理后有明显差异
nora_wt_rep1_mcool_file = "./Nora_WT/Nora_mESC_WT_HiC_rep1.nodups.pairs.1k.zoomify.mcool"
nora_wt_rep2_mcool_file = "./Nora_WT/Nora_mESC_WT_HiC_rep2.nodups.pairs.1k.zoomify.mcool"
ren_wt_rep1_mcool_file = "./Ren_WT/Ren_mESC_WT_HiC_rep1.nodups.pairs.1k.zoomify.mcool"
ren_wt_rep2_mcool_file = "./Ren_WT/Ren_mESC_WT_HiC_rep2.nodups.pairs.1k.zoomify.mcool"
# 生成 r10k cool 互作矩阵对象
nora_wt_rep1_r10k_clr = cooler.Cooler(f'{nora_wt_rep1_mcool_file}::/resolutions/{resolution}')
nora_wt_rep2_r10k_clr = cooler.Cooler(f'{nora_wt_rep2_mcool_file}::/resolutions/{resolution}')
ren_wt_rep1_r10k_clr = cooler.Cooler(f'{ren_wt_rep1_mcool_file}::/resolutions/{resolution}')
ren_wt_rep2_r10k_clr = cooler.Cooler(f'{ren_wt_rep2_mcool_file}::/resolutions/{resolution}')


## 读取 chrom view 文件

In [ ]:
# 读取 chrom view 文件
import pandas as pd
mm10_chr_view_df = pd.read_csv("./mm10.chrom.view.tsv", sep="\t", header=0)
print(mm10_chr_view_df.shape)
mm10_chr_view_df.head()


## 读取 CTCF_only summits 文件

In [ ]:
# 读取 mESC_WT_CTCFvsMBD3 CTCF_only summits 文件 
import pandas as pd
mESC_WT_CvM3_CTCF_only_summit_df = pd.read_csv("./mESC_WT_CvM3.CTCF_only_PS.sorted.bed", sep="\t", header=None)
mESC_WT_CvM3_CTCF_only_summit_df.columns = ["chrom", "start", "end", "fold_enrichment"]
print(mESC_WT_CvM3_CTCF_only_summit_df.shape)
mESC_WT_CvM3_CTCF_only_summit_df.head()


In [ ]:
# 为 summit 添加四分类标签
mESC_WT_CvM3_CTCF_only_summit_df['quartile_FC_score'] = pd.qcut(mESC_WT_CvM3_CTCF_only_summit_df['fold_enrichment'], 4, labels=False) + 1
print(mESC_WT_CvM3_CTCF_only_summit_df.shape)
mESC_WT_CvM3_CTCF_only_summit_df.head()

# 计算生成 expected_cis_df

In [ ]:
from cooltools import expected_cis
# Calculate expected interactions for chromosome arms
nora_wt_r10k_expected_cis_df = expected_cis(clr=nora_wt_r10k_clr, view_df=mm10_chr_view_df, ignore_diags=1, nproc=64)
nora_iaa_r10k_expected_cis_df = expected_cis(clr=nora_iaa_r10k_clr, view_df=mm10_chr_view_df, ignore_diags=1, nproc=64)
ren_wt_r10k_expected_cis_df = expected_cis(clr=ren_wt_r10k_clr, view_df=mm10_chr_view_df, ignore_diags=1, nproc=64)
ren_iaa_r10k_expected_cis_df = expected_cis(clr=ren_iaa_r10k_clr, view_df=mm10_chr_view_df, ignore_diags=1, nproc=64)
wn_bpk25_r10k_expected_cis_df = expected_cis(clr=wn_bpk25_r10k_clr, view_df=mm10_chr_view_df, ignore_diags=1, nproc=64)


In [ ]:
# Nora/Ren WT rep1/2 expected_cis_df
nora_wt_rep1_r10k_expected_cis_df = expected_cis(clr=nora_wt_rep1_r10k_clr, view_df=mm10_chr_view_df, ignore_diags=1, nproc=64)
nora_wt_rep2_r10k_expected_cis_df = expected_cis(clr=nora_wt_rep2_r10k_clr, view_df=mm10_chr_view_df, ignore_diags=1, nproc=64)
ren_wt_rep1_r10k_expected_cis_df = expected_cis(clr=ren_wt_rep1_r10k_clr, view_df=mm10_chr_view_df, ignore_diags=1, nproc=64)
ren_wt_rep2_r10k_expected_cis_df = expected_cis(clr=ren_wt_rep2_r10k_clr, view_df=mm10_chr_view_df, ignore_diags=1, nproc=64)


# CTCF_only_summits 无分组 pileup

## Nora_WT

In [ ]:
from coolpuppy import coolpup
nora_wt_r10k_clr_CTCF_only_pup = coolpup.pileup(clr=nora_wt_r10k_clr, 
                                            features=mESC_WT_CvM3_CTCF_only_summit_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=nora_wt_r10k_expected_cis_df,
                                            local=True,
                                            flank=300_000,
                                            min_diag=1,
                                            nproc=32)
nora_wt_r10k_clr_CTCF_only_pup

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=nora_wt_r10k_clr_CTCF_only_pup,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Nora_WT_r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Nora_WT_r10k.png")

## Nora_WT_rep1

In [ ]:
from coolpuppy import coolpup
nora_wt_rep1_r10k_clr_CTCF_only_pup = coolpup.pileup(clr=nora_wt_rep1_r10k_clr, 
                                            features=mESC_WT_CvM3_CTCF_only_summit_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=nora_wt_rep1_r10k_expected_cis_df,
                                            local=True,
                                            flank=300_000,
                                            min_diag=1,
                                            nproc=32)
nora_wt_rep1_r10k_clr_CTCF_only_pup

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=nora_wt_rep1_r10k_clr_CTCF_only_pup,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Nora_WT_rep1_r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Nora_WT_rep1_r10k.png")

## Nora_WT_rep2

In [ ]:
from coolpuppy import coolpup
nora_wt_rep2_r10k_clr_CTCF_only_pup = coolpup.pileup(clr=nora_wt_rep2_r10k_clr, 
                                            features=mESC_WT_CvM3_CTCF_only_summit_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=nora_wt_rep2_r10k_expected_cis_df,
                                            local=True,
                                            flank=300_000,
                                            min_diag=1,
                                            nproc=32)
nora_wt_rep2_r10k_clr_CTCF_only_pup

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=nora_wt_rep2_r10k_clr_CTCF_only_pup,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Nora_WT_rep2_r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Nora_WT_rep2_r10k.png")

## Nora_IAA

In [ ]:
from coolpuppy import coolpup
nora_iaa_r10k_clr_CTCF_only_pup = coolpup.pileup(clr=nora_iaa_r10k_clr, 
                                            features=mESC_WT_CvM3_CTCF_only_summit_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=nora_iaa_r10k_expected_cis_df,
                                            local=True,
                                            flank=300_000,
                                            min_diag=1,
                                            nproc=32)
nora_iaa_r10k_clr_CTCF_only_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=nora_iaa_r10k_clr_CTCF_only_pup,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Nora_IAA_r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Nora_IAA_r10k.png")


## Ren_WT

In [ ]:
from coolpuppy import coolpup
ren_wt_r10k_clr_CTCF_only_pup = coolpup.pileup(clr=ren_wt_r10k_clr, 
                                            features=mESC_WT_CvM3_CTCF_only_summit_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=ren_wt_r10k_expected_cis_df,
                                            local=True,
                                            flank=300_000,
                                            min_diag=1,
                                            nproc=32)
ren_wt_r10k_clr_CTCF_only_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ren_wt_r10k_clr_CTCF_only_pup,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Ren_WT_r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Ren_WT_r10k.png")


## Ren_WT_rep1

In [ ]:
from coolpuppy import coolpup
ren_wt_rep1_r10k_clr_CTCF_only_pup = coolpup.pileup(clr=ren_wt_rep1_r10k_clr, 
                                            features=mESC_WT_CvM3_CTCF_only_summit_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=ren_wt_rep1_r10k_expected_cis_df,
                                            local=True,
                                            flank=300_000,
                                            min_diag=1,
                                            nproc=32)
ren_wt_rep1_r10k_clr_CTCF_only_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ren_wt_rep1_r10k_clr_CTCF_only_pup,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Ren_WT_rep1_r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Ren_WT_rep1_r10k.png")


## Ren_WT_rep2

In [ ]:
from coolpuppy import coolpup
ren_wt_rep2_r10k_clr_CTCF_only_pup = coolpup.pileup(clr=ren_wt_rep2_r10k_clr, 
                                            features=mESC_WT_CvM3_CTCF_only_summit_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=ren_wt_rep2_r10k_expected_cis_df,
                                            local=True,
                                            flank=300_000,
                                            min_diag=1,
                                            nproc=32)
ren_wt_rep2_r10k_clr_CTCF_only_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ren_wt_rep2_r10k_clr_CTCF_only_pup,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Ren_WT_rep2_r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Ren_WT_rep2_r10k.png")


## Ren_IAA

In [ ]:
from coolpuppy import coolpup
ren_iaa_r10k_clr_CTCF_only_pup = coolpup.pileup(clr=ren_iaa_r10k_clr, 
                                            features=mESC_WT_CvM3_CTCF_only_summit_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=ren_iaa_r10k_expected_cis_df,
                                            local=True,
                                            flank=300_000,
                                            min_diag=1,
                                            nproc=32)
ren_iaa_r10k_clr_CTCF_only_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ren_iaa_r10k_clr_CTCF_only_pup,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Ren_IAA_r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Ren_IAA_r10k.png")


## WN_BPK25

In [ ]:
from coolpuppy import coolpup
wn_bpk25_r10k_clr_CTCF_only_pup = coolpup.pileup(clr=wn_bpk25_r10k_clr, 
                                            features=mESC_WT_CvM3_CTCF_only_summit_df,
                                            features_format='bed', 
                                            view_df=mm10_chr_view_df,
                                            expected_df=wn_bpk25_r10k_expected_cis_df,
                                            local=True,
                                            flank=300_000,
                                            min_diag=1,
                                            nproc=32)
wn_bpk25_r10k_clr_CTCF_only_pup


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=wn_bpk25_r10k_clr_CTCF_only_pup,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.WN_BPK25_r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.WN_BPK25_r10k.png")


## Nora_WT_rep12_merge_in_one

In [ ]:
nora_wt_rep12_CTCF_only_pup_merged_df = pd.concat([nora_wt_rep1_r10k_clr_CTCF_only_pup, nora_wt_rep2_r10k_clr_CTCF_only_pup], ignore_index=True)
nora_wt_rep12_CTCF_only_pup_merged_df


In [ ]:
nora_wt_rep12_CTCF_only_pup_merged_df["group"] = ["Nora_WT_rep1", "Nora_WT_rep2"]
nora_wt_rep12_CTCF_only_pup_merged_df


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=nora_wt_rep12_CTCF_only_pup_merged_df,
                cols="group",
                col_order=["Nora_WT_rep1", "Nora_WT_rep2"],
                colnames=["Nora_WT_rep1", "Nora_WT_rep2"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Nora_WT_merge_rep12_r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Nora_WT_merge_rep12_r10k.png")


## Ren_WT_rep12_merge_in_one

In [ ]:
ren_wt_rep12_CTCF_only_pup_merged_df = pd.concat([ren_wt_rep1_r10k_clr_CTCF_only_pup, ren_wt_rep2_r10k_clr_CTCF_only_pup], ignore_index=True)
ren_wt_rep12_CTCF_only_pup_merged_df


In [ ]:
ren_wt_rep12_CTCF_only_pup_merged_df["group"] = ["Ren_WT_rep1", "Ren_WT_rep2"]
ren_wt_rep12_CTCF_only_pup_merged_df


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ren_wt_rep12_CTCF_only_pup_merged_df,
                cols="group",
                col_order=["Ren_WT_rep1", "Ren_WT_rep2"],
                colnames=["Ren_WT_rep1", "Ren_WT_rep2"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Ren_WT_merge_rep12_r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Ren_WT_merge_rep12_r10k.png")


## Nora_Ren_WT_rep12_merge

In [ ]:
nora_ren_wt_rep12_CTCF_only_pup_merged_df = pd.concat([nora_wt_rep1_r10k_clr_CTCF_only_pup, nora_wt_rep2_r10k_clr_CTCF_only_pup, ren_wt_rep1_r10k_clr_CTCF_only_pup, ren_wt_rep2_r10k_clr_CTCF_only_pup], ignore_index=True)
nora_ren_wt_rep12_CTCF_only_pup_merged_df


In [ ]:
nora_ren_wt_rep12_CTCF_only_pup_merged_df["group"] = ["Nora_WT_rep1", "Nora_WT_rep2", "Ren_WT_rep1", "Ren_WT_rep2"]
nora_ren_wt_rep12_CTCF_only_pup_merged_df.insert(0, "WT_source", ["Nora", "Nora", "Ren", "Ren"] )
nora_ren_wt_rep12_CTCF_only_pup_merged_df.insert(1, "repeat", ["rep1", "rep2", "rep1", "rep2"] )
nora_ren_wt_rep12_CTCF_only_pup_merged_df

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=nora_ren_wt_rep12_CTCF_only_pup_merged_df,
                cols="repeat",
                col_order=["rep1", "rep2"],
                colnames=["rep1", "rep2"],
                rows="WT_source",
                row_order=["Nora", "Ren"],
                rownames=["Nora_WT", "Ren_WT"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.merge_Nora_Ren_WT_rep12_r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.merge_Nora_Ren_WT_rep12_r10k.png")


## Nora_WT_IAA_WN_BPK25_merge_in_one

In [ ]:
nora_wn_CTCF_only_pup_merged_df = pd.concat([nora_wt_r10k_clr_CTCF_only_pup, nora_iaa_r10k_clr_CTCF_only_pup, wn_bpk25_r10k_clr_CTCF_only_pup], ignore_index=True)
nora_wn_CTCF_only_pup_merged_df


In [ ]:
nora_wn_CTCF_only_pup_merged_df["group"] = ["Nora_WT", "Nora_IAA", "WN_BPK25"]
nora_wn_CTCF_only_pup_merged_df


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=nora_wn_CTCF_only_pup_merged_df,
                cols="group",
                col_order=["Nora_WT", "Nora_IAA", "WN_BPK25"],
                colnames=["Nora_WT", "Nora_IAA", "WN_BPK25"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Nora_WN_r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Nora_WN_r10k.png")


## Ren_WT_IAA_WN_BPK25_merge_in_one

In [ ]:
ren_wn_CTCF_only_pup_merged_df = pd.concat([ren_wt_r10k_clr_CTCF_only_pup, ren_iaa_r10k_clr_CTCF_only_pup, wn_bpk25_r10k_clr_CTCF_only_pup], ignore_index=True)
ren_wn_CTCF_only_pup_merged_df


In [ ]:
ren_wn_CTCF_only_pup_merged_df["group"] = ["Ren_WT", "Ren_IAA", "WN_BPK25"]
ren_wn_CTCF_only_pup_merged_df


In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=ren_wn_CTCF_only_pup_merged_df,
                cols="group",
                col_order=["Ren_WT", "Ren_IAA", "WN_BPK25"],
                colnames=["Ren_WT", "Ren_IAA", "WN_BPK25"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Ren_WN_r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Ren_WN_r10k.png")


## all experiment merge in one

In [ ]:
nora_wn_ren_CTCF_only_pup_merged_df = pd.concat([nora_wt_r10k_clr_CTCF_only_pup, nora_iaa_r10k_clr_CTCF_only_pup, wn_bpk25_r10k_clr_CTCF_only_pup, ren_iaa_r10k_clr_CTCF_only_pup, ren_wt_r10k_clr_CTCF_only_pup], ignore_index=True)
nora_wn_ren_CTCF_only_pup_merged_df


In [ ]:
nora_wn_ren_CTCF_only_pup_merged_df["group"] = ["Nora_WT", "Nora_IAA", "WN_BPK25", "Ren_IAA", "Ren_WT"]
nora_wn_ren_CTCF_only_pup_merged_df

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=nora_wn_ren_CTCF_only_pup_merged_df,
                cols="group",
                col_order=["Nora_WT", "Nora_IAA", "WN_BPK25", "Ren_IAA", "Ren_WT"],
                colnames=["Nora_WT", "Nora_IAA", "WN_BPK25", "Ren_IAA", "Ren_WT"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Nora_WN_Ren_r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Nora_WN_Ren_r10k.png")


## Nora_WT_rep2_vs_rep1

In [ ]:
# nora_wt_rep1_r10k_clr_CTCF_only_pup
# nora_wt_rep2_r10k_clr_CTCF_only_pup
from coolpuppy.lib.puputils import divide_pups
CTCF_only_pup_nora_wt_rep2_vs_rep1 = divide_pups(nora_wt_rep2_r10k_clr_CTCF_only_pup, nora_wt_rep1_r10k_clr_CTCF_only_pup)
CTCF_only_pup_nora_wt_rep2_vs_rep1

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=CTCF_only_pup_nora_wt_rep2_vs_rep1,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            vmax=1.1,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Nora_WT_rep2_vs_rep1.r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Nora_WT_rep2_vs_rep1.r10k.png")

## Ren_WT_rep2_vs_rep1

In [ ]:
# ren_wt_rep1_r10k_clr_CTCF_only_pup
# ren_wt_rep2_r10k_clr_CTCF_only_pup
from coolpuppy.lib.puputils import divide_pups
CTCF_only_pup_ren_wt_rep2_vs_rep1 = divide_pups(ren_wt_rep2_r10k_clr_CTCF_only_pup, ren_wt_rep1_r10k_clr_CTCF_only_pup)
CTCF_only_pup_ren_wt_rep2_vs_rep1

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=CTCF_only_pup_ren_wt_rep2_vs_rep1,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            vmax=1.2,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Ren_WT_rep2_vs_rep1.r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Ren_WT_rep2_vs_rep1.r10k.png")

## Nora_IAA_vs_WT

In [ ]:
# nora_wt_r10k_clr_CTCF_only_pup
# nora_iaa_r10k_clr_CTCF_only_pup
from coolpuppy.lib.puputils import divide_pups
CTCF_only_pup_nora_iaa_vs_wt = divide_pups(nora_iaa_r10k_clr_CTCF_only_pup, nora_wt_r10k_clr_CTCF_only_pup)
CTCF_only_pup_nora_iaa_vs_wt

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=CTCF_only_pup_nora_iaa_vs_wt,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Nora_IAA_vs_WT.r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Nora_IAA_vs_WT.r10k.png")

## Ren_IAA_vs_WT

In [ ]:
# ren_wt_r10k_clr_CTCF_only_pup
# ren_iaa_r10k_clr_CTCF_only_pup
from coolpuppy.lib.puputils import divide_pups
CTCF_only_pup_ren_iaa_vs_wt = divide_pups(ren_iaa_r10k_clr_CTCF_only_pup, ren_wt_r10k_clr_CTCF_only_pup)
CTCF_only_pup_ren_iaa_vs_wt

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=CTCF_only_pup_ren_iaa_vs_wt,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Ren_IAA_vs_WT.r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Ren_IAA_vs_WT.r10k.png")

## WN_BPK25_vs_Nora_WT

In [ ]:
# nora_wt_r10k_clr_CTCF_only_pup
# wn_bpk25_r10k_clr_CTCF_only_pup
from coolpuppy.lib.puputils import divide_pups
CTCF_only_pup_wn_bpk25_vs_nora_wt = divide_pups(wn_bpk25_r10k_clr_CTCF_only_pup, nora_wt_r10k_clr_CTCF_only_pup)
CTCF_only_pup_wn_bpk25_vs_nora_wt

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=CTCF_only_pup_wn_bpk25_vs_nora_wt,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.WN_BPK25_vs_Nora_WT.r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.WN_BPK25_vs_Nora_WT.r10k.png")

## WN_BPK25_vs_Ren_WT

In [ ]:
# ren_wt_r10k_clr_CTCF_only_pup
# wn_bpk25_r10k_clr_CTCF_only_pup
from coolpuppy.lib.puputils import divide_pups
CTCF_only_pup_wn_bpk25_vs_ren_wt = divide_pups(wn_bpk25_r10k_clr_CTCF_only_pup, ren_wt_r10k_clr_CTCF_only_pup)
CTCF_only_pup_wn_bpk25_vs_ren_wt

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=CTCF_only_pup_wn_bpk25_vs_ren_wt,
            score=True,
            sym=True,
            cmap='coolwarm',
            scale='log',
            height=5,
            font_scale=1.2,
            plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.WN_BPK25_vs_Ren_WT.r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.WN_BPK25_vs_Ren_WT.r10k.png")


## merge Nora_WN 3 divide pileup in one

In [ ]:
# CTCF_only_pup_nora_wt_rep2_vs_rep1
# CTCF_only_pup_nora_iaa_vs_wt
# CTCF_only_pup_wn_bpk25_vs_nora_wt
CTCF_only_pup_divide_merged_df = pd.concat([CTCF_only_pup_nora_wt_rep2_vs_rep1, CTCF_only_pup_nora_iaa_vs_wt, CTCF_only_pup_wn_bpk25_vs_nora_wt], ignore_index=True)
CTCF_only_pup_divide_merged_df

In [ ]:
CTCF_only_pup_divide_merged_df.insert(0, "groups", ["Nora_WT_rep2vs1", "Nora_IAA_vs_WT", "WN_BPK25_vs_Nora_WT"] )
CTCF_only_pup_divide_merged_df

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=CTCF_only_pup_divide_merged_df,
                cols="groups",
                col_order=["Nora_WT_rep2vs1", "Nora_IAA_vs_WT", "WN_BPK25_vs_Nora_WT"],
                colnames=["Nora_WT_rep2vs1", "Nora_IAA_vs_WT", "WN_BPK25_vs_Nora_WT"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Nora_WN_r10k.merge_3divide.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Nora_WN_r10k.merge_3divide.png")


## merge Ren_WN 3 divide pileup in one

In [ ]:
# CTCF_only_pup_ren_wt_rep2_vs_rep1
# CTCF_only_pup_ren_iaa_vs_wt
# CTCF_only_pup_wn_bpk25_vs_ren_wt
CTCF_only_pup_divide_merged_df = pd.concat([CTCF_only_pup_ren_wt_rep2_vs_rep1, CTCF_only_pup_ren_iaa_vs_wt, CTCF_only_pup_wn_bpk25_vs_ren_wt], ignore_index=True)
CTCF_only_pup_divide_merged_df

In [ ]:
CTCF_only_pup_divide_merged_df.insert(0, "groups", ["Ren_WT_rep2vs1", "Ren_IAA_vs_WT", "WN_BPK25_vs_Ren_WT"] )
CTCF_only_pup_divide_merged_df

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=CTCF_only_pup_divide_merged_df,
                cols="groups",
                col_order=["Ren_WT_rep2vs1", "Ren_IAA_vs_WT", "WN_BPK25_vs_Ren_WT"],
                colnames=["Ren_WT_rep2vs1", "Ren_IAA_vs_WT", "WN_BPK25_vs_Ren_WT"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Ren_WN_r10k.merge_3divide.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Ren_WN_r10k.merge_3divide.png")


## merge Nora_Ren_WN 6 divide pileup in one

In [ ]:
CTCF_only_pup_divide_merged_df = pd.concat([CTCF_only_pup_nora_wt_rep2_vs_rep1, CTCF_only_pup_ren_wt_rep2_vs_rep1, CTCF_only_pup_nora_iaa_vs_wt, CTCF_only_pup_ren_iaa_vs_wt, CTCF_only_pup_wn_bpk25_vs_nora_wt, CTCF_only_pup_wn_bpk25_vs_ren_wt], ignore_index=True)
CTCF_only_pup_divide_merged_df


In [ ]:
CTCF_only_pup_divide_merged_df.insert(0, "comp_group", ["WT_rep2vs1", "WT_rep2vs1", "IAA_vs_WT", "IAA_vs_WT", "BPK25_vs_WT", "BPK25_vs_WT"] )
CTCF_only_pup_divide_merged_df.insert(1, "WT_source", ["Nora", "Ren", "Nora", "Ren", "Nora", "Ren"] )
CTCF_only_pup_divide_merged_df

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=CTCF_only_pup_divide_merged_df,
                cols="comp_group",
                col_order=["WT_rep2vs1", "IAA_vs_WT", "BPK25_vs_WT"],
                colnames=["WT_rep2vs1", "IAA_vs_WT", "BPK25_vs_WT"],
                rows='WT_source',
                row_order=["Nora", "Ren"],
                rownames=["use_Nora_WT", "use_Ren_WT"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=5,
                font_scale=1.2,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.merge_6comparation.r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.merge_6comparation.r10k.png")


## merge Nora_Ren_WN 4 divide pileup in one (rm rep2v1)

In [ ]:
CTCF_only_pup_divide_merged_df = pd.concat([CTCF_only_pup_nora_iaa_vs_wt, CTCF_only_pup_ren_iaa_vs_wt, CTCF_only_pup_wn_bpk25_vs_nora_wt, CTCF_only_pup_wn_bpk25_vs_ren_wt], ignore_index=True)
CTCF_only_pup_divide_merged_df

In [ ]:
CTCF_only_pup_divide_merged_df.insert(0, "comp_group", ["IAA_vs_WT", "IAA_vs_WT", "BPK25_vs_WT", "BPK25_vs_WT"] )
CTCF_only_pup_divide_merged_df.insert(1, "WT_source", ["Nora", "Ren", "Nora", "Ren"] )
CTCF_only_pup_divide_merged_df

In [ ]:
import matplotlib.pyplot as plt
from coolpuppy import plotpup
plotpup.plot(pupsdf=CTCF_only_pup_divide_merged_df,
                cols="comp_group",
                col_order=["IAA_vs_WT", "BPK25_vs_WT"],
                colnames=["IAA_vs_WT", "BPK25_vs_WT"],
                rows='WT_source',
                row_order=["Nora", "Ren"],
                rownames=["use_Nora_WT", "use_Ren_WT"],
                score=True,
                sym=True,
                cmap='coolwarm',
                scale='log',
                height=6,
                font_scale=1.1,
                plot_ticks=True)
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.merge_4comparation.r10k.pdf")
plt.savefig("coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.merge_4comparation.r10k.png")


# 制作结果汇总 PPT

In [ ]:
! ls -1 coolpuppy_*_for_mESC_WT_CvM3_CTCF_only_BS*r10k*.pdf

In [ ]:
! ls -1 coolpuppy_*_for_mESC_WT_CvM3_CTCF_only_BS*r10k*.png

In [ ]:
from PIL import Image
from pptx import Presentation
from pptx.util import Pt, Cm
from pptx.dml.color import RGBColor
from pptx.enum.text import PP_ALIGN

CTCF_only_BS_noC_Nora_WT_rep1_r10k_png = "coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Nora_WT_rep1_r10k.png"
CTCF_only_BS_noC_Nora_WT_rep2_r10k_png = "coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Nora_WT_rep2_r10k.png"
CTCF_only_BS_noC_Ren_WT_rep1_r10k_png = "coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Ren_WT_rep1_r10k.png"
CTCF_only_BS_noC_Ren_WT_rep2_r10k_png = "coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Ren_WT_rep2_r10k.png"

CTCF_only_BS_noC_Nora_WT_r10k_png = "coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Nora_WT_r10k.png"
CTCF_only_BS_noC_Nora_IAA_r10k_png = "coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Nora_IAA_r10k.png"
CTCF_only_BS_noC_Ren_WT_r10k_png = "coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Ren_WT_r10k.png"
CTCF_only_BS_noC_Ren_IAA_r10k_png = "coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Ren_IAA_r10k.png"
CTCF_only_BS_noC_WN_BPK25_r10k_png = "coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.WN_BPK25_r10k.png"

def add_text_anno(slide, anno_text, left, top, width, height):
    # 插入文本框
    text_box = slide.shapes.add_textbox(left, top, width, height)
    # 设置文本框背景颜色为黄色
    text_box.fill.solid()
    text_box.fill.fore_color.rgb = RGBColor(255, 230, 153)
    # 在文本框中添加内容并设置格式
    text_box.text_frame.text = anno_text
    text_box.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
    # 设置字体大小和加粗
    text_font = text_box.text_frame.paragraphs[0].font
    text_font.name = '微软雅黑'
    text_font.size = Pt(8)
    text_font.bold = True

# 创建 PPT 并插入图片
my_ppt = Presentation()
# 设置幻灯片尺寸为 16:9 (25.4cm *14.28cm)
my_ppt.slide_width = 9144000
my_ppt.slide_height = 5143500
# slide1
slide1 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
# pic = slide.shapes.add_picture(png_file, left, top, width, height)
slide1.shapes.add_picture(CTCF_only_BS_noC_Nora_WT_rep1_r10k_png, Cm(3.21), Cm(0), Cm(7.14), Cm(7.14))
slide1.shapes.add_picture(CTCF_only_BS_noC_Nora_WT_rep2_r10k_png, Cm(15.07), Cm(0), Cm(7.14), Cm(7.14))
slide1.shapes.add_picture(CTCF_only_BS_noC_Ren_WT_rep1_r10k_png, Cm(3.21), Cm(7.14), Cm(7.14), Cm(7.14))
slide1.shapes.add_picture(CTCF_only_BS_noC_Ren_WT_rep2_r10k_png, Cm(15.07), Cm(7.14), Cm(7.14), Cm(7.14))
# 插入文本注释
add_text_anno(slide1, "CTCF_only_BS_Nora_WT_rep1_pileup_plot", Cm(3.21), Cm(0.05), Cm(7.20), Cm(0.6))
add_text_anno(slide1, "CTCF_only_BS_Nora_WT_rep2_pileup_plot", Cm(15.07), Cm(0.05), Cm(7.20), Cm(0.6))
add_text_anno(slide1, "CTCF_only_BS_Ren_WT_rep1_pileup_plot", Cm(3.21), Cm(7.19), Cm(7.20), Cm(0.6))
add_text_anno(slide1, "CTCF_only_BS_Ren_WT_rep2_pileup_plot", Cm(15.07), Cm(7.19), Cm(7.20), Cm(0.6))

# slide2
slide2 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
# pic = slide.shapes.add_picture(png_files[pi-1], left, top, width, height)
pic = slide2.shapes.add_picture(CTCF_only_BS_noC_Nora_WT_r10k_png, Cm(0.7), Cm(0.05), Cm(7.125), Cm(7.125))
pic = slide2.shapes.add_picture(CTCF_only_BS_noC_Nora_IAA_r10k_png, Cm(9.14), Cm(0.05), Cm(7.125), Cm(7.125))
pic = slide2.shapes.add_picture(CTCF_only_BS_noC_WN_BPK25_r10k_png, Cm(17.625), Cm(0.05), Cm(7.125), Cm(7.125))
pic = slide2.shapes.add_picture(CTCF_only_BS_noC_Ren_WT_r10k_png, Cm(0.7), Cm(7.14), Cm(7.125), Cm(7.125))
pic = slide2.shapes.add_picture(CTCF_only_BS_noC_Ren_IAA_r10k_png, Cm(9.14), Cm(7.14), Cm(7.125), Cm(7.125))
# 插入文本注释
add_text_anno(slide2, "CTCF_only_BS_Nora_WT_r10k_pileup_plot", Cm(0.1), Cm(0.05), Cm(8), Cm(0.6))
add_text_anno(slide2, "CTCF_only_BS_Nora_IAA_r10k_pileup_plot", Cm(8.65), Cm(0.05), Cm(8), Cm(0.6))
add_text_anno(slide2, "CTCF_only_BS_WN_BPK25_r10k_pileup_plot", Cm(17.3), Cm(0.05), Cm(8), Cm(0.6))
add_text_anno(slide2, "CTCF_only_BS_Ren_WT_r10k_pileup_plot", Cm(0.1), Cm(7.24), Cm(8), Cm(0.6))
add_text_anno(slide2, "CTCF_only_BS_Ren_IAA_r10k_pileup_plot", Cm(8.65), Cm(7.24), Cm(8), Cm(0.6))

# 保存 PPT
my_ppt.save('8.15_coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS.use_Nora_WN_Ren_mcool.r10k.summary.pptx')


In [ ]:
CTCF_only_BS_noC_Nora_WT_rep12_merge_r10k_png = "coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Nora_WT_merge_rep12_r10k.png"
CTCF_only_BS_noC_Ren_WT_rep12_merge_r10k_png = "coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Ren_WT_merge_rep12_r10k.png"
CTCF_only_BS_noC_merge_Nora_Ren_WT_rep12_r10k_png = "coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.merge_Nora_Ren_WT_rep12_r10k.png"

CTCF_only_BS_noC_Nora_WN_r10k_png = "coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Nora_WN_r10k.png"
CTCF_only_BS_noC_Ren_WN_r10k_png = "coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Ren_WN_r10k.png"
CTCF_only_BS_noC_Nora_WN_Ren_r10k_png = "coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Nora_WN_Ren_r10k.png"

# 打开 PPT 并插入图片
my_ppt = Presentation('8.15_coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS.use_Nora_WN_Ren_mcool.r10k.summary.pptx')
# slide3
slide3 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
slide3.shapes.add_picture(CTCF_only_BS_noC_Nora_WT_rep12_merge_r10k_png, Cm(0), Cm(1.44), Cm(11.1), Cm(5.55))
slide3.shapes.add_picture(CTCF_only_BS_noC_Ren_WT_rep12_merge_r10k_png, Cm(0), Cm(8.04),  Cm(11.1), Cm(5.55))
slide3.shapes.add_picture(CTCF_only_BS_noC_merge_Nora_Ren_WT_rep12_r10k_png, Cm(11.12), Cm(0), Cm(14.28), Cm(14.28))
# 插入文本注释
add_text_anno(slide3, "CTCF_only_BS_Nora_WT_rep12_r10k_pileup_plot", Cm(0), Cm(0.91), Cm(8.9), Cm(0.60))
add_text_anno(slide3, "CTCF_only_BS_Ren_WT_rep12_r10k_pileup_plot", Cm(0), Cm(7.53), Cm(8.9), Cm(0.60))
add_text_anno(slide3, "CTCF_only_BS_Nora_Ren_WT_rep12_r10k_pileup_plot", Cm(13.81), Cm(0.39), Cm(8.9), Cm(0.60))

# slide4
slide4 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
slide4.shapes.add_picture(CTCF_only_BS_noC_Nora_WN_r10k_png, Cm(6.23), Cm(0.02), Cm(12.94), Cm(4.31))
slide4.shapes.add_picture(CTCF_only_BS_noC_Ren_WN_r10k_png, Cm(6.23), Cm(4.54),  Cm(12.94), Cm(4.31))
slide4.shapes.add_picture(CTCF_only_BS_noC_Nora_WN_Ren_r10k_png, Cm(0), Cm(9.04), Cm(25.4), Cm(5.08))
# 插入文本注释
add_text_anno(slide4, "CTCF_only_BS_Nora_WN_r10k_pileup_plot", Cm(0.28), Cm(1.92), Cm(7), Cm(0.60))
add_text_anno(slide4, "CTCF_only_BS_Ren_WN_r10k_pileup_plot", Cm(0.28), Cm(6.44), Cm(7), Cm(0.60))
add_text_anno(slide4, "CTCF_only_BS_Nora_WN_Ren_r10k_pileup_plot", Cm(0.28), Cm(8.86), Cm(7), Cm(0.60))

# 保存 PPT
my_ppt.save('8.15_coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS.use_Nora_WN_Ren_mcool.r10k.summary.pptx')


In [ ]:
CTCF_only_BS_noC_Nora_WT_rep2_vs_rep1 = "coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Nora_WT_rep2_vs_rep1.r10k.png"
CTCF_only_BS_noC_Ren_WT_rep2_vs_rep1 = "coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Ren_WT_rep2_vs_rep1.r10k.png"
CTCF_only_BS_noC_Nora_IAA_vs_WT = "coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Nora_IAA_vs_WT.r10k.png"
CTCF_only_BS_noC_Ren_IAA_vs_WT = "coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Ren_IAA_vs_WT.r10k.png"
CTCF_only_BS_noC_WN_BPK25_vs_Nora_WT = "coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.WN_BPK25_vs_Nora_WT.r10k.png"
CTCF_only_BS_noC_WN_BPK25_vs_Ren_WT = "coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.WN_BPK25_vs_Ren_WT.r10k.png"

# 打开 PPT 并插入图片
my_ppt = Presentation('8.15_coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS.use_Nora_WN_Ren_mcool.r10k.summary.pptx')
# slide5
slide5 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
# pic = slide.shapes.add_picture(png_files[pi-1], left, top, width, height)
slide5.shapes.add_picture(CTCF_only_BS_noC_Nora_WT_rep2_vs_rep1, Cm(0.53), Cm(0), Cm(7.14), Cm(7.14))
slide5.shapes.add_picture(CTCF_only_BS_noC_Nora_IAA_vs_WT, Cm(9.08), Cm(0), Cm(7.14), Cm(7.14))
slide5.shapes.add_picture(CTCF_only_BS_noC_WN_BPK25_vs_Nora_WT, Cm(17.73), Cm(0), Cm(7.14), Cm(7.14))
slide5.shapes.add_picture(CTCF_only_BS_noC_Ren_WT_rep2_vs_rep1, Cm(0.53), Cm(7.14), Cm(7.14), Cm(7.14))
slide5.shapes.add_picture(CTCF_only_BS_noC_Ren_IAA_vs_WT, Cm(9.08), Cm(7.14), Cm(7.14), Cm(7.14))
slide5.shapes.add_picture(CTCF_only_BS_noC_WN_BPK25_vs_Ren_WT, Cm(17.73), Cm(7.14), Cm(7.14), Cm(7.14))
# 插入文本注释
add_text_anno(slide5, "CTCF_only_BS_Nora_WT_rep2_vs_rep1", Cm(0.53), Cm(0), Cm(7.14), Cm(0.60))
add_text_anno(slide5, "CTCF_only_BS_Nora_IAA_vs_WT", Cm(9.08), Cm(0), Cm(7.14), Cm(0.60))
add_text_anno(slide5, "CTCF_only_BS_WN_BPK25_vs_Nora_WT", Cm(17.73), Cm(0), Cm(7.14), Cm(0.60))
add_text_anno(slide5, "CTCF_only_BS_Ren_WT_rep2_vs_rep1", Cm(0.53), Cm(7.14), Cm(7.14), Cm(0.60))
add_text_anno(slide5, "CTCF_only_BS_Ren_IAA_vs_WT", Cm(9.08), Cm(7.14), Cm(7.14), Cm(0.60))
add_text_anno(slide5, "CTCF_only_BS_WN_BPK25_vs_Ren_WT", Cm(17.73), Cm(7.14), Cm(7.14), Cm(0.60))

# 保存 PPT
my_ppt.save('8.15_coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS.use_Nora_WN_Ren_mcool.r10k.summary.pptx')


In [ ]:
CTCF_only_BS_noC_Nora_WN_3comparation = "coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Nora_WN_r10k.merge_3divide.png"
CTCF_only_BS_noC_Ren_WN_3comparation = "coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.Ren_WN_r10k.merge_3divide.png"
CTCF_only_BS_noC_Nora_Ren_WN_6comparation = "coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.merge_6comparation.r10k.png"
CTCF_only_BS_noC_Nora_Ren_WN_4comparation = "coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS_noClassified.merge_4comparation.r10k.png"

# 打开 PPT 并插入图片
my_ppt = Presentation('8.15_coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS.use_Nora_WN_Ren_mcool.r10k.summary.pptx')
# slide6
slide6 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
# pic = slide.shapes.add_picture(png_files[pi-1], left, top, width, height)
slide6.shapes.add_picture(CTCF_only_BS_noC_Nora_WN_3comparation, Cm(1.98), Cm(0), Cm(21.43), Cm(7.14))
slide6.shapes.add_picture(CTCF_only_BS_noC_Ren_WN_3comparation, Cm(1.98), Cm(7.14), Cm(21.43), Cm(7.14))
# 插入文本注释
add_text_anno(slide6, "CTCF_only_BS_Nora_WN_3comparation", Cm(0), Cm(0.05), Cm(12.7), Cm(0.6))
add_text_anno(slide6, "CTCF_only_BS_Ren_WN_3comparation", Cm(0), Cm(7.14), Cm(12.7), Cm(0.6))

# slide7
slide7 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
slide7.shapes.add_picture(CTCF_only_BS_noC_Nora_Ren_WN_6comparation, Cm(1.98), Cm(0), Cm(21.43), Cm(14.28))
# 插入文本注释
add_text_anno(slide7, "CTCF_only_BS_Nora_Ren_WN_6comparation", Cm(0), Cm(0.05), Cm(12.7), Cm(0.6))

# slide8
slide8 = my_ppt.slides.add_slide(my_ppt.slide_layouts[6])
# PPT 页面指定区域插入图片
slide8.shapes.add_picture(CTCF_only_BS_noC_Nora_Ren_WN_4comparation, Cm(5.56), Cm(0), Cm(14.48), Cm(14.28))
# 插入文本注释
add_text_anno(slide8, "CTCF_only_BS_Nora_Ren_WN_4comparation", Cm(0), Cm(0.05), Cm(12.7), Cm(0.6))

# 保存 PPT
my_ppt.save('8.15_coolpuppy_pileup_plot_for_mESC_WT_CvM3_CTCF_only_BS.use_Nora_WN_Ren_mcool.r10k.summary.pptx')


# 移动相关结果到特定文件夹

In [ ]:
%%bash
mkdir coolpuppy_pileup_plot_for_mESC_CvM3_CTCF_only_BS_use_r10k
mv coolpuppy_*_for_mESC_WT_CvM3_CTCF_only_BS*r10k*.pdf ./coolpuppy_pileup_plot_for_mESC_CvM3_CTCF_only_BS_use_r10k/
mv coolpuppy_*_for_mESC_WT_CvM3_CTCF_only_BS*r10k*.png ./coolpuppy_pileup_plot_for_mESC_CvM3_CTCF_only_BS_use_r10k/
